<a href="https://colab.research.google.com/github/jihyun923/city_change/blob/main/precleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import cv2

img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/pro/data/input/jj202512.tif')
h, w, _ = img.shape

crop_w, crop_h = 2200, 1160

center_x, center_y = w//2, h//2

x1 = center_x - crop_w // 2
y1 = center_y - crop_h // 2
x2 = center_x + crop_w // 2
y2 = center_y + crop_h // 2

crop = img[y1:y2, x1:x2]

cv2.imwrite('crop_jj202512.png', crop)

True

In [ ]:
import rasterio
import numpy as np
import cv2
import os

file_list = {
    "jeonju_2015_crop":   "/content/drive/MyDrive/Colab Notebooks/pro/data/input/jeounju_2015.tif",
    "jeonju_2020_crop":   "/content/drive/MyDrive/Colab Notebooks/pro/data/input/jeounju_2020.tif",
    "jeonju_2025_crop":   "/content/drive/MyDrive/Colab Notebooks/pro/data/input/jeounju_2025.tif",
    "jeonju_202512_crop": "/content/drive/MyDrive/Colab Notebooks/pro/data/input/jeounju_202512.tif",
#     "namwon_2015_crop":   "/content/drive/MyDrive/Colab Notebooks/pro/data/input/namwon_2015.tif",
#     "namwon_2021_crop":   "/content/drive/MyDrive/Colab Notebooks/pro/data/input/namwon_2021.tif",
#     "namwon_2025_crop":   "/content/drive/MyDrive/Colab Notebooks/pro/data/input/namwon_2025.tif",
#     "namwon_202512_crop": "/content/drive/MyDrive/Colab Notebooks/pro/data/input/namwon_202512.tif",
}

crop_w, crop_h = 2320, 2010
# crop_w, crop_h = 4530, 2940


out_dir = "/content/drive/MyDrive/Colab Notebooks/pro/data/ddia"
os.makedirs(out_dir, exist_ok=True)

for name, path in file_list.items():
    with rasterio.open(path) as src:
        arr = src.read()  # (C,H,W)

    # 3채널로 맞추기
    if arr.shape[0] >= 3:
        arr = arr[:3]
    else:
        arr = np.repeat(arr, 3, axis=0)

    img = np.transpose(arr, (1, 2, 0))  # (H,W,3)
    h, w, _ = img.shape
    print(name, "orig shape:", h, w)

    # 1) 원본이 너무 작으면 에러 던지기
    if w < crop_w or h < crop_h:
        raise ValueError(f"{name}: 원본({w}x{h})이 crop크기({crop_w}x{crop_h})보다 작음")

    center_x, center_y = w // 2, h // 2

    # 2) 원하는 크기를 유지한 채로 x1,x2,y1,y2 조정
    x1 = center_x - crop_w // 2
    y1 = center_y - crop_h // 2

    # 좌표가 음수로 안 나가게 조정
    x1 = max(0, min(x1, w - crop_w))
    y1 = max(0, min(y1, h - crop_h))

    x2 = x1 + crop_w
    y2 = y1 + crop_h

    crop = img[y1:y2, x1:x2]
    print(name, "crop shape:", crop.shape)  # <- 항상 (crop_h, crop_w, 3) 나와야 정상

    save_path = os.path.join(out_dir, f"{name}.tif")
    ok = cv2.imwrite(save_path, cv2.cvtColor(crop, cv2.COLOR_RGB2BGR))
    print("Saved:", save_path, "->", ok)

jeonju_2015_crop orig shape: 2130 2587
jeonju_2015_crop crop shape: (2010, 2320, 3)
Saved: /content/drive/MyDrive/Colab Notebooks/pro/data/ddia/jeonju_2015_crop.tif -> True
jeonju_2020_crop orig shape: 2019 2344
jeonju_2020_crop crop shape: (2010, 2320, 3)
Saved: /content/drive/MyDrive/Colab Notebooks/pro/data/ddia/jeonju_2020_crop.tif -> True
jeonju_2025_crop orig shape: 2042 2321
jeonju_2025_crop crop shape: (2010, 2320, 3)
Saved: /content/drive/MyDrive/Colab Notebooks/pro/data/ddia/jeonju_2025_crop.tif -> True
jeonju_202512_crop orig shape: 2076 2434
jeonju_202512_crop crop shape: (2010, 2320, 3)
Saved: /content/drive/MyDrive/Colab Notebooks/pro/data/ddia/jeonju_202512_crop.tif -> True


In [ ]:
## png -> jpg

from PIL import Image

png_path = "/content/drive/MyDrive/Colab Notebooks/pro/data/input/namwon_202512_crop.png"
jpg_path = "/content/drive/MyDrive/Colab Notebooks/pro/data/input/namwon_202512_crop.jpg"

img = Image.open(png_path).convert("RGB")
img.save(jpg_path, quality=90)

In [ ]:
## 학습한 데이터와 입력 데이터 차이 가지고 방사보정

import rasterio
import numpy as np

STATS_FILE  = "/content/drive/MyDrive/Colab Notebooks/pro/data/val/urban_stats_ref.npz"
NEW_IMG     = "/content/drive/MyDrive/Colab Notebooks/pro/data/input/jeonju_2015_crop.tif"
OUT_MATCHED = "/content/drive/MyDrive/Colab Notebooks/pro/data/input/new_jj_2015.tif"

# 1) 참조 색상 통계 불러오기
stats = np.load(STATS_FILE)
ref_mean = stats["mean"]   # (C_ref,)
ref_std  = stats["std"]    # (C_ref,)

# 2) 새 이미지 읽기
with rasterio.open(NEW_IMG) as src:
    new_arr = src.read().astype(np.float32)   # (C_new,H,W)
    profile = src.profile

C_ref = ref_mean.shape[0]
C_new, H, W = new_arr.shape

# --- 🔴 채널 수 맞추기 (공통 채널까지만 사용) ---
C = min(C_ref, C_new)
ref_mean = ref_mean[:C]
ref_std  = ref_std[:C]
new_arr  = new_arr[:C]

print("[INFO] new_arr shape:", new_arr.shape)
print("[INFO] ref_mean:", ref_mean)
print("[INFO] ref_std :", ref_std)

# 3) 새 이미지 전체 통계
new_flat = new_arr.reshape(C, -1)
new_mean = new_flat.mean(axis=1)   # (C,)
new_std  = new_flat.std(axis=1)    # (C,)

print("[INFO] new_mean:", new_mean)
print("[INFO] new_std :", new_std)

# 4) mean/std 매칭
eps = 1e-6
scale = ref_std / (new_std + eps)           # (C,)
shift = ref_mean - new_mean * scale         # (C,)

matched = new_arr * scale[:, None, None] + shift[:, None, None]

# 5) TIFF(float32)로 저장
matched = matched.astype(np.float32)

profile.update(
    driver="GTiff",
    dtype=rasterio.float32,
    count=C,
    compress="lzw",
)

with rasterio.open(OUT_MATCHED, "w", **profile) as dst:
    dst.write(matched)

print("[SAVE] color matched ->", OUT_MATCHED)

/usr/local/lib/python3.12/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


[INFO] new_arr shape: (3, 2010, 2320)
[INFO] ref_mean: [147.34619 149.35698 150.67061]
[INFO] ref_std : [41.254745 41.73509  37.71091 ]
[INFO] new_mean: [85.638275 77.92219  65.948524]
[INFO] new_std : [35.512028 30.800308 28.727314]


/usr/local/lib/python3.12/dist-packages/rasterio/__init__.py:366: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  dataset = writer(


[SAVE] color matched -> /content/drive/MyDrive/Colab Notebooks/pro/data/input/new_jj_2015.tif


In [ ]:
## 그냥 방사보

from skimage.exposure import match_histograms
import rasterio
import numpy as np

REF_IMG = "/content/drive/MyDrive/Colab Notebooks/pro/data/val/img/LC_JL_SN_34602_003_20210522_FGT.tif"
NEW_IMG = "/content/drive/MyDrive/Colab Notebooks/pro/data/input/2512.tif"
OUT_MATCHED = "/content/drive/MyDrive/Colab Notebooks/pro/data/input/2512.jpg"

with rasterio.open(REF_IMG) as ref:
    ref_arr = ref.read().astype(np.float32)      # (C,H,W)

# 2. 새 이미지 불러오기 + profile 저장
with rasterio.open(NEW_IMG) as src:
    new_arr = src.read().astype(np.float32)
    profile = src.profile                        # 📌 profile 여기서 저장해야 함

# 3. 채널 수 맞추기 (최소공배수)
C = min(ref_arr.shape[0], new_arr.shape[0])
ref_arr = ref_arr[:C]
new_arr = new_arr[:C]

# 4. 히스토그램 매칭
matched = np.zeros_like(new_arr, dtype=np.float32)
for c in range(C):
    matched[c] = match_histograms(new_arr[c], ref_arr[c])

# 5. 새 파일로 저장
with rasterio.open(OUT_MATCHED, "w", **profile) as dst:
    dst.write(matched.astype(profile["dtype"]))

print("[SAVE] matched hist →", OUT_MATCHED)